In [58]:
import pandas as pd
import optbinning as ob
import sklearn as sl
from optbinning import OptimalBinning, BinningProcess
import matplotlib.pyplot as plt
#import seaborn as sns
import numpy as np
import ydata_profiling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Lasso
from sklearn import metrics
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from ydata_profiling import ProfileReport
import category_encoders as ce
from sklearn.neighbors import LocalOutlierFactor
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler


<h2>DIY</h2>

*Reading data*

In [110]:
read = pd.read_csv('D:/maga_1/ML/data/external/diy.csv', index_col= False)
print(read)
diy = read[["Monetary_A", "Frequency"]]
# print(sample)
diy_clean = diy[diy["Frequency"] > 1]

print(diy_clean)

          ClientID  Recency  Frequency  Monetary_Q  Monetary_A
0      client13166      682          2          23        2705
1       client1239       35         43         219       42161
2      client30041      190         25         133       16057
3      client36276      289          4          12        4614
4      client14136      217          6          36       35870
...            ...      ...        ...         ...         ...
42741  client35593        1          1           3       12797
42742  client22822        1          1           4         536
42743  client23138        1          1          33        6269
42744  client32589        1          1          46       12372
42745  client25258        1          1           5         945

[42746 rows x 5 columns]
       Monetary_A  Frequency
0            2705          2
1           42161         43
2           16057         25
3            4614          4
4           35870          6
...           ...        ...
42656        53

**Standartization**

In [111]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(diy_clean)
diy_clean = X_scaled

<h3>Поиск аномалий и фильтрация</h3>

In [112]:
lof = LocalOutlierFactor(n_neighbors=20)
diy_pred_lof = lof.fit_predict(diy_clean)
num_anomalies_lof = sum(diy_pred_lof == -1)
print("Found anomalies")
print(num_anomalies_lof)

diy_filtered = diy_clean[diy_pred_lof == 1]
anomalies = diy_clean[diy_pred_lof == -1]

Found anomalies
75


<h3>Поиск с novelty</h3>

In [113]:
lof_novelty = LocalOutlierFactor(n_neighbors=20, novelty=True)
lof_novelty.fit(diy_filtered)
diy_pred_novelty = lof_novelty.predict(anomalies)
matches = sum(diy_pred_novelty == -1)
print("Matches LOF novelty")
print(matches)

Matches LOF novelty
75


<h3>Isolation forest</h3>

In [115]:
iso_forest = IsolationForest(n_estimators=100, random_state=None, contamination= 0.01)
diy_pred_iso = iso_forest.fit_predict(diy_clean)
num_anomalies_iso = sum(diy_pred_iso == -1)
print("Isolation Forest matches")
print(num_anomalies_iso)


Isolation Forest matches
297


<h3>Метрики</h3>

In [65]:
matches = sum(diy_pred_novelty == -1)
print("Anomalies matched")
freq_matches = matches / len(diy_clean)
print("Frequency LOF")
print(freq_matches)
matches = sum(diy_pred_iso == -1)
print("Frequency Isolation Forest")
freq_matches = matches / len(diy_clean)
print(freq_matches)

Anomalies matched
Frequency LOF
0.03800983705289925
Frequency Isolation Forest
0.19945126643691238


<h2>Bank</h2>

*Reading data*

In [ ]:
read = pd.read_csv('D:/maga_1/ML/data/external/banks.csv', index_col= False, encoding='cp1251')
print(read)
banks = read[["Assents",  "OwnCapital",  "IndFunds",  "NBSLoans",  "IndLoans"]]
# # print(sample)
# diy_clean = diy[diy["Frequency"] > 1]


                 Bank  Assents  OwnCapital  IndFunds  NBSLoans  IndLoans
0          «Авангард»   122109       20440     35443     32728      3319
1             «Аверс»   110741       24410     34918     13613      4924
2             «Агора»     1114         356       274       351       206
3    «Агропромкредит»    18774        2332     12047      6484       903
4           «Агророс»     7917        1157      3564      1909       492
..                ...      ...         ...       ...       ...       ...
366      Ю БИ ЭС Банк     7117        4164         0        43         0
367     Юг-Инвестбанк    12504        1431      7407      6637      2238
368               ЮМК     5024        2117         0      2585       235
369    Юникредит-банк  1315550      197369    267400    665780    170748
370       Яринтербанк     2820         362      1307      1732       221

[371 rows x 6 columns]
     Assents  OwnCapital  IndFunds  NBSLoans  IndLoans  row_number
0     122109       20440     3544

<h3>Поиск аномалий и фильтрация</h3>

In [116]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(banks)
banks = X_scaled
# banks['row_number'] = range(1, len(banks) + 1)
# print(banks)
lof = LocalOutlierFactor(n_neighbors=20)
banks_pred_lof = lof.fit_predict(banks)
# lof_pred_num = banks['row_number']
# lof_pred_num["pred"] = banks_pred_lof
# print(lof_pred_num)
num_anomalies_lof = sum(banks_pred_lof == -1)
print("Found anomalies")
print(num_anomalies_lof)

banks_filtered = banks[banks_pred_lof == 1]
anomalies = banks[banks_pred_lof == -1]

Found anomalies
44


<h3>Поиск с novelty</h3>

In [117]:
lof_novelty = LocalOutlierFactor(n_neighbors=20, novelty=True)
lof_novelty.fit(banks_filtered)
banks_pred_novelty = lof_novelty.predict(anomalies)

matches = sum(banks_pred_novelty == -1)
print("Macthes LOF novelty")
print(matches)

Macthes LOF novelty
44


<h3>Isolation forest</h3>

In [118]:
iso_forest = IsolationForest(n_estimators=100, random_state=None)
banks_pred_iso = iso_forest.fit_predict(banks)
num_anomalies_iso = sum(banks_pred_iso == -1)
print("Isolation forest matches")
print(num_anomalies_iso)


Isolation forest matches
29


<h3>Метрики</h3>

In [109]:
matches = sum(banks_pred_novelty == -1)
print("Anomalies matched")
freq_matches = matches / len(banks)
print("Frequency LOF")
print(freq_matches)
matches = sum(banks_pred_iso == -1)
print("Frequency Isolation Forest")
freq_matches = matches / len(banks)
print(freq_matches)

# # Подсчёт совпадающих и новых выбросов
# # Совпадают те, что аномальные и в обоих режимах (-1)
# common_anomalies = np.sum((banks_pred_lof == -1) & (banks_pred_novelty == -1))
# # Новые выбросы: объекты, которые в режиме обычного LOF были нормальными (1), а в режиме Novelty определены как выбросы (-1)
# new_anomalies = np.sum((banks_pred_lof == 1) & (banks_pred_novelty == -1))
# print(f"Совпадающих выбросов (оба режима): {common_anomalies}")
# print(f"Новых выбросов (только Novelty=True): {new_anomalies}")

Anomalies matched
Frequency LOF
0.20754716981132076
Frequency Isolation Forest
0.07547169811320754
